## 5. Convert CSV to PARQUET File

Code learned and followed from AAI 540 Lab 2

In [2]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
from pyathena import connect

In [4]:
# Set S3 path to Parquet data
s3_path_parquet = "s3://{}/AAI-540_Predictive-Maintenance-for-Pharmaceutical-Manufacturing-Equipment/data/parquet".format(bucket)

# Set Athena parameters
database_name = "dsoawsfp"
table_name_csv = "predictive_maintenance_csv"
table_name_parquet = "predictive_maintenance_parquet"

In [5]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{}/athena/staging".format(bucket)

In [6]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [7]:
# SQL statement to execute
statement = """CREATE TABLE IF NOT EXISTS {}.{}
WITH (
    format = 'PARQUET',
    external_location = '{}',
    parquet_compression = 'SNAPPY'
) AS
SELECT   date,
         device,
         failure,
         metric1,
         metric2,
         metric3,
         metric4,
         metric5,
         metric6,
         metric7,
         metric8,
         metric9
FROM {}.{}""".format(
    database_name, table_name_parquet, s3_path_parquet, database_name, table_name_csv
)

print(statement)

CREATE TABLE IF NOT EXISTS dsoawsfp.predictive_maintenance_parquet
WITH (
    format = 'PARQUET',
    external_location = 's3://sagemaker-us-east-1-807494057176/AAI-540_Predictive-Maintenance-for-Pharmaceutical-Manufacturing-Equipment/data/parquet',
    parquet_compression = 'SNAPPY'
) AS
SELECT   date,
         device,
         failure,
         metric1,
         metric2,
         metric3,
         metric4,
         metric5,
         metric6,
         metric7,
         metric8,
         metric9
FROM dsoawsfp.predictive_maintenance_csv


In [8]:
print(f"Parquet table {table_name_parquet} created successfully and stored at {s3_path_parquet}.")

Parquet table predictive_maintenance_parquet created successfully and stored at s3://sagemaker-us-east-1-807494057176/AAI-540_Predictive-Maintenance-for-Pharmaceutical-Manufacturing-Equipment/data/parquet.


In [9]:
import pandas as pd

pd.read_sql(statement, conn)

/tmp/ipykernel_91/3803073958.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


,rows


In [10]:
statement = "MSCK REPAIR TABLE {}.{}".format(database_name, table_name_parquet)

print(statement)

MSCK REPAIR TABLE dsoawsfp.predictive_maintenance_parquet


In [11]:
df = pd.read_sql(statement, conn)
df.head(5)

/tmp/ipykernel_91/2446512133.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


""


In [21]:
# Define the SQL statement with casting to timestamp
statement = """SELECT * FROM {}.{}
    WHERE date >= CAST('2015-01-01' AS timestamp)
    AND date <= CAST('2015-11-02' AS timestamp)
    LIMIT 100""".format(database_name, table_name_parquet)

# Execute the query using pandas
df_result = pd.read_sql(statement, conn)

# Display the first few rows of the result
print(df_result.head())


/tmp/ipykernel_91/2785210363.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql(statement, conn)


        date    device  failure      metric1  metric2  metric3  metric4  \
0 2015-01-01  S1F01085        0  215630672.0     55.0      0.0     52.0   
1 2015-01-01  S1F0166B        0   61370680.0      0.0      3.0      0.0   
2 2015-01-01  S1F01E6Y        0  173295968.0      0.0      0.0      0.0   
3 2015-01-01  S1F01JE0        0   79694024.0      0.0      0.0      0.0   
4 2015-01-01  S1F01R2B        0  135970480.0      0.0      0.0      0.0   

   metric5   metric6  metric7  metric8  metric9  
0      6.0  407438.0      0.0      0.0      7.0  
1      6.0  403174.0      0.0      0.0      0.0  
2     12.0  237394.0      0.0      0.0      0.0  
3      6.0  410186.0      0.0      0.0      0.0  
4     15.0  313173.0      0.0      0.0      3.0  


In [22]:
df = pd.read_sql(statement, conn)
df

/tmp/ipykernel_91/8222894.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,date,device,failure,metric1,metric2,metric3,metric4,metric5,metric6,metric7,metric8,metric9
0,2015-01-01,S1F01085,0,215630672.0,55.0,0.0,52.0,6.0,407438.0,0.0,0.0,7.0
1,2015-01-01,S1F0166B,0,61370680.0,0.0,3.0,0.0,6.0,403174.0,0.0,0.0,0.0
2,2015-01-01,S1F01E6Y,0,173295968.0,0.0,0.0,0.0,12.0,237394.0,0.0,0.0,0.0
3,2015-01-01,S1F01JE0,0,79694024.0,0.0,0.0,0.0,6.0,410186.0,0.0,0.0,0.0
4,2015-01-01,S1F01R2B,0,135970480.0,0.0,0.0,0.0,15.0,313173.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,2015-01-01,S1F0GKFX,0,107148272.0,0.0,0.0,0.0,65.0,223170.0,0.0,0.0,0.0
96,2015-01-01,S1F0GKL6,0,234778096.0,0.0,0.0,0.0,90.0,221425.0,0.0,0.0,0.0
97,2015-01-01,S1F0GPFZ,0,53071760.0,0.0,0.0,0.0,8.0,195262.0,0.0,0.0,0.0
98,2015-01-01,S1F0GPXY,0,22380768.0,0.0,0.0,0.0,8.0,196205.0,0.0,0.0,0.0


In [23]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>